In [54]:
%pip install --upgrade pip
%pip install boto3>=1.33.2 --force-reinstall --quiet
%pip install botocore>=1.33.2 --force-reinstall --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.29 requires botocore==1.34.29, but you have botocore 1.34.58 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.29 requires botocore==1.34.29, but you have botocore 1.34.58 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0

In [66]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import boto3
import json
import base64
import time

bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')

In [2]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens,top_p,temp,system='',if_print=0):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system
        }  
    )  
    start_time = time.time()
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    end_time = time.time()
    elapsed_time = end_time - start_time
    if if_print == 1:
        print("----------------------------------------- OutPut -----------------------------------------")
        print("Elapsed time: ", elapsed_time, "seconds")
    response_body = json.loads(response.get('body').read())

    return response_body

In [23]:
with open('test2.png', "rb") as image_file:
    content_image = base64.b64encode(image_file.read()).decode('utf8')

In [25]:
message_mm=[{ "role": "user","content": [{"type": "image","source": { "type": "base64","media_type": \
                                                                     "image/jpeg","data": content_image}},\
                                                                     {"type": "text","text": "请从图像中提取广告口号，仅输出广告口号，广告口号不需要翻译，并以<Response>开始，</Response>结束的格式输出"}]}]

In [26]:
data = generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=message_mm,max_tokens=4096,temp=0.5,top_p=0.9,if_print=1)

text_value = data['content'][0]['text']
response_value = text_value[text_value.find('<Response>') + len('<Response>'):text_value.find('</Response>')]

print(response_value)

----------------------------------------- OutPut -----------------------------------------
Elapsed time:  2.000746726989746 seconds
All right, babe


In [52]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'msg_01J4TcGGmFxQztykcNKHDHf3',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': "Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy on the atomic and subatomic scale. It is a mathematical framework that provides a way to understand and predict the properties and interactions of particles at the quantum level.\n\nSome key principles and concepts of quantum mechanics include:\n\n1. Wave-particle duality: Particles can exhibit both wave-like and particle-like properties, depending on the experimental situation.\n\n2. Quantum state: The state of a quantum system is described by a wave function, which contains all the information about the system's properties.\n\n3. Quantum superposition: A quantum system can exist in a superposition of multiple states simultaneously, until it is measured or interacts with the environment.\n\n4. Uncertainty principle: There is a fundamental limit to the precision wi

In [22]:
#Messages API with system prompt
# messages = [{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]
user_message1 =  {"role": "user", "content": "查看上文后分两次回复，第一次仅能回复'🎵'第二次直接按照要求继续描写，符合字数要求。"}
assistant_message1 =  {"role": "assistant", "content": "🎵"}
user_message2 =  {"role": "user", "content": "请继续"}
assistant_message2 =  {"role": "assistant", "content": "🎵"}
message_mm={ "role": "user","content": [{"type": "image","source": { "type": "base64","media_type": \
                                                                     "image/jpeg","data": content_image}},\
                                                                     {"type": "text","text": "请从图像中提取广告口号，仅输出广告口号，广告口号不需要翻译，并以<Response>开始，</Response>结束的格式输出"}]}
messages = [user_message1, assistant_message1,user_message2, assistant_message2, message_mm]

system = "You are a text reviewer, checking for grammar errors"
generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=2048,temp=0.01,top_p=0.2,system=system)

{'id': 'msg_bdrk_01F8hj5YSi8XgUAvkZ17gb7Y',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text', 'text': '🎵\n\n<Response>你好</Response>'}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 435, 'output_tokens': 16}}